In [674]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
from dash.dash_table import DataTable
from datetime import date
import psycopg2
import base64
import io
import pandas as pd


In [675]:
def get_db_connection():
    return psycopg2.connect(
        host="localhost",
        dbname = "school_fee_db",
        user = "uthkarsh",
        password = "Ruthwik081@")

In [676]:
APP_STYLE = {
    'font-family': 'Arial, sans-serif',
    'margin': '0 auto',
    'padding': '20px',
    'max-width': '800px',
    'background-color': '#ffffff',  # Softer white background
    'box-shadow': '0 4px 8px rgba(0, 0, 0, 0.1)',
    'border-radius': '10px',
    'padding-bottom': '30px',
}

HEADER_STYLE = {
    'text-align': 'center',
    'padding': '15px 0',
    'background-color': '#0066cc',  # Calming blue header
    'color': 'white',
    'border-radius': '10px 10px 0 0',
    'font-size': '24px',
    'font-weight': 'bold'
}

SUB_HEADER_STYLE = {
    'text-align': 'center',
    'font-size': '24px',
    'font-weight': 'bold'    
}

NAV_BAR_STYLE = {
    'text-align': 'center',
    'margin': '15px 0',
}

NAV_LINK_STYLE = {
    'margin': '0 10px',
    'padding': '10px 20px',
    'background-color': '#0066cc',  # Matches header
    'color': 'white',
    'border-radius': '5px',
    'text-decoration': 'none',
    'font-weight': 'bold',
    'box-shadow': '0 2px 4px rgba(0, 0, 0, 0.2)',  # Adds depth
}

SECTION_STYLE = {
    'padding': '15px 20px',
    'display': 'flex',
    'flex-direction': 'column',
    'align-items': 'center',
}

BUTTON_STYLE = {
    'background-color': '#0066cc',  # Matches header and nav
    'color': 'white',
    'border': 'none',
    'padding': '10px 30px',
    'border-radius': '5px',
    'cursor': 'pointer',
    'font-size': '16px',
    'margin-top': '15px',
    'box-shadow': '0 2px 4px rgba(0, 0, 0, 0.2)',  # Subtle shadow for depth
}

INPUT_STYLE = {
    'margin': '10px 0',
    'padding': '12px 15px',
    'width': '100%',
    'border': '1px solid #ccc',
    'border-radius': '5px',
    'font-size': '14px',
    'box-shadow': 'inset 0 1px 3px rgba(0, 0, 0, 0.1)',  # Adds dimension
}

OUTPUT_STYLE = {
    "marginTop": "20px",
    "padding": "10px",
    "border": "1px solid #ccc",
    "borderRadius": "5px",
    "backgroundColor": "#f9f9f9",
    "fontSize": "16px"
}

DROPDOWN_STYLE = {
    'margin': '10px 0',
    'padding': '12px 15px',
    'width': '100%',
    'height': '50%',
    'border': '1px solid #ccc',
    'border-radius': '1px',
    'font-size': '14px',
    'background-color': '#f9f9f9',  # Softer background to match input
    'box-shadow': 'inset 0 1px 3px rgba(0, 0, 0, 0.1)',  # Same as input
}

DATE_STYLE = {
    'display': 'flex',
    'justify-content': 'space-between',
    'gap': '15px',  # Balanced spacing between date fields
    'width': '100%',
}

DATE_INPUT_STYLE = {
    'padding': '12px 15px',
    'flex': '1',  # Ensures even width for both date inputs
    'border': '1px solid #ccc',
    'border-radius': '5px',
    'font-size': '14px',
    'box-shadow': 'inset 0 1px 3px rgba(0, 0, 0, 0.1)',
}


In [677]:
def generate_input(id, placeholder, input_type='text'):
    return dcc.Input(id=id, type=input_type, placeholder=placeholder, style=INPUT_STYLE)

def generate_dropdown(id, options, placeholder):
    return dcc.Dropdown(id=id, options=options, placeholder=placeholder, style=DROPDOWN_STYLE)

In [678]:
def student_details_layout():
    input_fields = [
        {'id': 'admission_number', 'type': 'number', 'placeholder': 'Admission Number', 'value': None},
        {'id': 'aadhar_number', 'type': 'number', 'placeholder': 'Aadhar Number', 'value': None},
        {'id': 'StudentName', 'type': 'text', 'placeholder': 'Student Name', 'value': ''},
        {'id': 'FatherName', 'type': 'text', 'placeholder': "Father's Name", 'value': ''},
        {'id': 'contact_number', 'type': 'tel', 'placeholder': 'Contact Number', 'value': ''},
        {'id': 'village', 'type': 'text', 'placeholder': 'Village', 'value': ''}
    ]

    date_pickers = [
        {'id': 'dob', 'placeholder': 'Date of Birth'},
        {'id': 'doj', 'placeholder': 'Date of Joining'}
    ]

    return html.Div([
        html.H1("Student Details Entry", style=HEADER_STYLE),
        html.Div([
            *[dcc.Input(id=field['id'], type=field['type'], placeholder=field['placeholder'], value=field['value'], style=INPUT_STYLE) for field in input_fields],
            dcc.Dropdown(
                id='gender',
                options=[{'label': label, 'value': label} for label in ['Boy', 'Girl']],
                placeholder="Gender",
                style=DROPDOWN_STYLE
            ),
            html.Div([
                *[dcc.DatePickerSingle(id=date['id'], display_format='DD/MM/YYYY', placeholder=date['placeholder'], style=DATE_INPUT_STYLE) for date in date_pickers]
            ], style=DATE_STYLE),
            html.Button('Submit', id='student-details-submit-button', n_clicks=0, style=BUTTON_STYLE),
            dcc.Upload(
                id='upload-csv',
                children=html.Button('Import from CSV', style=BUTTON_STYLE),
                accept='.csv',
                style={'margin-top': '10px'}
            ),
        ], style=SECTION_STYLE),
        html.A("Back to Home", href='/', style={**NAV_LINK_STYLE, 'display': 'inline-block', 'margin-top': '20px'})
    ], style=APP_STYLE)

In [679]:
# Page 2: Student Class Details Layout
def student_class_details_layout():
    return html.Div([
        html.H1("Student Class Details Entry", style=HEADER_STYLE),
        html.Div([
            generate_input('admission_number', 'Admission Number', 'number'),
            generate_input('roll_number', 'Roll Number', 'number'),
            generate_input('photo_id', 'Photo ID', 'number'),
            dcc.Dropdown(
                id='class_no', 
                options=[{'label': f'Class {i}', 'value': i} for i in range(1, 13)],
                placeholder="Class",
                style=DROPDOWN_STYLE
            ),
            generate_input('section', 'Section', 'text'),
            generate_input('current_year', 'Year', 'number'),
            dcc.Dropdown(id='enrolled', options=[
                {'label': 'Yes', 'value': 'Yes'},
                {'label': 'No', 'value': 'No'}
            ], placeholder="Currently Enrolled", style=DROPDOWN_STYLE),
            dcc.Dropdown(id='language', options=[
                {'label': 'Telugu', 'value': 'Telugu'},
                {'label': 'Hindi', 'value': 'Hindi'},
                {'label': 'Sanskrit', 'value': 'Sanskrit'}
            ], placeholder="Language", style=DROPDOWN_STYLE),
            dcc.Dropdown(id='vocational', options=[
                {'label': 'Agriculture', 'value': 'Agriculture'},
                {'label': 'Artificial Intelligence', 'value': 'AI'},
                {'label': 'Physical Activity Trainer', 'value': 'PAT'},
                {'label': 'Tourism', 'value': 'Tourism'}
            ], placeholder="Vocational", style=DROPDOWN_STYLE),
            html.Button('Submit', id='student-class-submit-button', n_clicks=0, style=BUTTON_STYLE),
        ], style=SECTION_STYLE),
        html.A("Back to Home", href='/', style={**NAV_LINK_STYLE, 'display': 'inline-block', 'margin-top': '20px'})
    ], style=APP_STYLE)

In [680]:
# Page 3: Fee Details Layout
def fee_details_layout():
    return html.Div([
        html.H1("Fee Details Entry", style=HEADER_STYLE),
        html.Div([
            dcc.Input(id='admission_number', type='number', placeholder='Admission Number', style=INPUT_STYLE),
            dcc.Input(id='year', type='number', placeholder='Year', style=INPUT_STYLE),
            dcc.Input(id='school_fee', type='number', placeholder='School Fee', style=INPUT_STYLE),
            dcc.Dropdown(
                id='school_fee_concession_reason',
                options=[
                    {'label': reason, 'value': reason} for reason in [
                        'Staff', 'Sibling', 'OTP', 'TF', 'FP', 'EC', 'SC', 'General'
                    ]
                ],
                placeholder="School Fee Concession Reason",
                style=DROPDOWN_STYLE
            ),
            dcc.Dropdown(
                id='transport_used',
                options=[
                    {'label': 'Yes', 'value': 'Yes'},
                    {'label': 'No', 'value': 'No'}
                ],
                placeholder="Transport Used",
                style=DROPDOWN_STYLE
            ),
            dcc.Input(id='transport_route', type='number', placeholder='Route Number', style=INPUT_STYLE, disabled=True),
            dcc.Input(id='transport_fee', type='number', placeholder='Transport Fee', style=INPUT_STYLE, disabled=True),
            dcc.Input(id='transport_fee_concession', type='text', placeholder='Transport Fee Concession', style=INPUT_STYLE, disabled=True),
            dcc.Input(id='application_fee', type='number', placeholder='Application Fee', style=INPUT_STYLE),
            html.Button('Submit', id='fee-details-submit-button', n_clicks=0, style=BUTTON_STYLE),
            ], style=SECTION_STYLE),
        html.A("Back to Home", href='/', style={**NAV_LINK_STYLE, 'display': 'inline-block', 'margin-top': '20px'})
    ], style=APP_STYLE)

In [681]:
# Page 4: Fee Details Layout
def transport_details_layout():
    return html.Div([
        html.H1("Transport Details Entry", style=HEADER_STYLE),
        html.Div([
            dcc.Input(id='pickup_point', type='text', placeholder='Pick-Up Point', style=INPUT_STYLE),
            dcc.Input(id='transport_route', type='number', placeholder='Route Number', style=INPUT_STYLE),
           html.Button('Submit', id='transport-details-submit-button', n_clicks=0, style=BUTTON_STYLE),
            ], style=SECTION_STYLE),
        html.A("Back to Home", href='/', style={**NAV_LINK_STYLE, 'display': 'inline-block', 'margin-top': '20px'})
    ], style=APP_STYLE)

In [682]:
# Page 5: Fee Payment Details Layout
def fee_type_selection_layout():
    input_fields = [
        {'id': 'paid', 'type': 'number', 'placeholder': 'Fee Paid'},
        {'id': 'due', 'type': 'number', 'placeholder': 'Fee Due'},
        {'id': 'receipt_number', 'type': 'number', 'placeholder': 'Receipt Number'}
    ]
    return html.Div([
        html.H1("Fee Payment Details Entry", style=HEADER_STYLE),
        html.Div([
            dcc.Input(id='admission_number', type='number', placeholder='Admission Number', style=INPUT_STYLE),
            dcc.Input(id='year', type='number', placeholder='Year', style=INPUT_STYLE),
            dcc.Dropdown(
                id='fee_type',
                options=[
                    {'label': 'School Fee', 'value': 'school_fee'},
                    {'label': 'Transport Fee', 'value': 'transport_fee'},
                    {'label': 'Application Fee', 'value': 'application_fee'}
                ],
                value=None,  # Default to None
                placeholder="Select Fee Type",
                style=DROPDOWN_STYLE
            ),
            dcc.Dropdown(
                id='fee_term',
                options=[
                    {'label': 'Term 1', 'value': 'term_1'},
                    {'label': 'Term 2', 'value': 'term_2'},
                    {'label': 'Term 3', 'value': 'term_3'}
                ],
                value=None,  # Default to None
                placeholder="Select Fee Term",
                style=DROPDOWN_STYLE
            ),
            *[dcc.Input(id = field['id'], type = field['type'], placeholder = field['placeholder'], style=INPUT_STYLE) for field in input_fields],
            html.H2("Date paid", style= SUB_HEADER_STYLE),
            dcc.DatePickerSingle(
                id='paid_date',
                display_format='DD/MM/YYYY',
                placeholder="Date Paid",
                date=date.today(),  # Set the default date to today
                style={'margin-top': '10px'}
            ),
            html.Button('Submit', id='feepayment-details-submit-button', n_clicks=0, style=BUTTON_STYLE)
        ], style=SECTION_STYLE),
        html.Div(id='fee-content'),
        html.A("Back to Home", href='/', style={**NAV_LINK_STYLE, 'display': 'inline-block', 'margin-top': '20px'})
    ], style=APP_STYLE)

In [683]:
# Page 6: View Student Records Layout
def view_student_details_layout():
    return html.Div([
        html.H1("Student Details", style=HEADER_STYLE),
        html.Div([
            dcc.Input(
                id='admission-number',
                type='number',
                placeholder="Enter Admission Number",
                style=INPUT_STYLE
            ),
            dcc.Dropdown(
                id='student-year-dropdown',
                options=[{'label': str(year), 'value': year} for year in range(2020, 2025)],
                placeholder="Select Year",
                style=DROPDOWN_STYLE
            ),
            html.Button("Fetch Details", id='fetch-student-details', style=BUTTON_STYLE),
            html.Div(id='student-details-output', style=OUTPUT_STYLE)
        ], style=SECTION_STYLE),
        html.A("Back to Home", href='/', style={**NAV_LINK_STYLE, 'display': 'inline-block', 'margin-top': '20px'})
    ], style=APP_STYLE)


In [684]:
# Page 7: View Class Records Layout
def class_details_layout():
    return html.Div([
        html.H1("Class Details", style=HEADER_STYLE),
        html.Div([
            dcc.Dropdown(
                id='class-dropdown',
                options=[{'label': f'Class {i}', 'value': str(i)} for i in range(1, 13)],
                placeholder="Select Class",
                style=DROPDOWN_STYLE
            ),
            dcc.Dropdown(
                id='year-dropdown',
                options=[{'label': str(year), 'value': year} for year in range(2020, 2025)],
                placeholder="Select Year",
                style=DROPDOWN_STYLE
            ),
            html.Button("Fetch Details", id='fetch-class-details', style=BUTTON_STYLE),
            html.Div(id='class-details-output', style=OUTPUT_STYLE)
        ], style=SECTION_STYLE),
        html.A("Back to Home", href='/', style={**NAV_LINK_STYLE, 'display': 'inline-block', 'margin-top': '20px'})
    ], style=APP_STYLE)


In [685]:
# Define app with URL routing
app = dash.Dash(__name__, suppress_callback_exceptions=True)

# Create navigation bar
def navigation_bar():
    return html.Div([
        html.A("Student Details", href='/student-details', style=NAV_LINK_STYLE),
        html.A("Student Class Details", href='/student-class-details', style=NAV_LINK_STYLE),
        html.A("Transport Details", href='/transport-details', style=NAV_LINK_STYLE),
        html.A("Fee Details", href='/fee-details', style=NAV_LINK_STYLE),
        html.A("Fee Payment Details", href='/fee-payment-details', style=NAV_LINK_STYLE),
        html.A("View Student Details", href='/view-student-details', style=NAV_LINK_STYLE),
        html.A("View Class-Wise Details", href='/view-class-details', style=NAV_LINK_STYLE),
    ], style=NAV_BAR_STYLE)

# Set app layout with navigation bar and page content
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),  # Tracks current URL
    navigation_bar(),
    html.Div(id='page-content')
])

# Update page content based on URL
@app.callback(
    Output('page-content', 'children'),
    [Input('url', 'pathname')]
)

def display_page(pathname):
    if pathname == '/student-details':
        return student_details_layout()
    elif pathname == '/student-class-details':
        return student_class_details_layout()
    elif pathname == '/transport-details':
        return transport_details_layout()
    elif pathname == '/fee-details':
        return fee_details_layout()
    elif pathname == '/view-student-details':
        return view_student_details_layout()
    elif pathname == '/view-class-details':
        return class_details_layout()
    elif pathname == '/fee-payment-details':
        return fee_type_selection_layout()
    elif pathname == '/':
        return html.Div([
            html.H1("Welcome to the Student Management System", style=HEADER_STYLE),
            html.P("Use the navigation bar above to explore different pages.", style={'text-align': 'center'})
        ], style=APP_STYLE)
    else:
        return html.Div([
            html.H1('404: Page not found', style=HEADER_STYLE),
            html.P('The page you are looking for does not exist.', style={'text-align': 'center'})
        ], style=APP_STYLE)

In [686]:
# Toggle transport inputs based on transport usage selection
@app.callback(
    [
        Output('transport_fee', 'disabled'),
        Output('transport_fee_concession', 'disabled'),
        Output('transport_route', 'disabled'), 
    ],
    Input('transport_used', 'value')
)
def toggle_transport_inputs(transport_used):
    if transport_used == 'No':
        return True, True, True  # Disable fields
    return False, False, False  # Enable fields

In [687]:
@app.callback(
    Output('student-details-submit-button', 'children'),
    inputs=[
        Input('student-details-submit-button', 'n_clicks'),
        State('admission_number', 'value'),
        State('StudentName', 'value'),
        State('FatherName', 'value'),
        State('gender', 'value'),
        State('aadhar_number', 'value'),
        State('dob', 'date'),
        State('doj', 'date'),
        State('contact_number', 'value'),
        State('village', 'value'),
    ]
)
def upload_student_data(n_clicks, admission_number, student_name, guardian_name, gender, aadhar_number, date_of_birth, date_of_joining, contact_number, village):
    if n_clicks > 0:
        try:
            # Validate required fields
            if not all([admission_number, student_name, guardian_name, gender, date_of_birth, date_of_joining]):
                return "Please fill all required fields"

            # Connect to the database
            conn = get_db_connection()
            cursor = conn.cursor()

            # Check if admission number already exists
            check_query = "SELECT COUNT(*) FROM Student WHERE admission_number = %s or aadhar_number = %s"
            cursor.execute(check_query, (admission_number, aadhar_number))
            count = cursor.fetchone()[0]

            if count > 0:
                return "Admission number or Aadhar number already exists. Please use unique values."

            # SQL query to insert data
            insert_query = """
                INSERT INTO Student (
                    admission_number, student_name, guardian_name, gender, aadhar_number,
                    date_of_birth, date_of_joining, contact_number, village
                )
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                ON CONFLICT (admission_number) DO NOTHING
            """

            # Execute query with the submitted data
            cursor.execute(insert_query, (
                admission_number, student_name, guardian_name, gender,
                aadhar_number, date_of_birth, date_of_joining, contact_number, village
            ))

            # Commit changes
            conn.commit()

            # Close connection
            cursor.close()
            conn.close()

            return "Data submitted successfully!"

        except Exception as e:
            return f"Error: {str(e)}"

    return "Submit"

In [688]:
@app.callback(
    Output('upload-csv', 'children'),
    Input('upload-csv', 'contents')
)
def import_csv(contents):
    if contents:
        try:
            content_type, content_string = contents.split(',')
            decoded = base64.b64decode(content_string)
            df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))

            # Connect to the database
            conn = get_db_connection()
            cursor = conn.cursor()

            for _, row in df.iterrows():
                admission_number = row['admission_number']
                student_name = row['student_name']
                guardian_name = row['guardian_name']
                gender = row['gender']
                aadhar_number = row['aadhar_number']
                date_of_birth = row['date_of_birth']
                date_of_joining = row['date_of_joining']
                contact_number = row['contact_number']
                village = row['village']

                # SQL query to insert data
                insert_query = """
                    INSERT INTO Student (
                        admission_number, student_name, guardian_name, gender, aadhar_number,
                        date_of_birth, date_of_joining, contact_number, village
                    )
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                    ON CONFLICT (admission_number) DO NOTHING
                """
                cursor.execute(insert_query, (
                    admission_number, student_name, guardian_name, gender,
                    aadhar_number, date_of_birth, date_of_joining, contact_number, village
                ))

            # Commit changes
            conn.commit()
            cursor.close()
            conn.close()

            return "CSV data imported successfully!"

        except Exception as e:
            return f"Error processing file: {str(e)}"

    return "Import from CSV"

In [689]:
@app.callback(
    Output('student-class-submit-button', 'children'),
    inputs=[
        Input('student-class-submit-button', 'n_clicks'),
        State('admission_number', 'value'),
        State('roll_number', 'value'),
        State('photo_id', 'value'),
        State('class_no', 'value'),
        State('section', 'value'),
        State('current_year', 'value'),
        State('enrolled', 'value'),
        State('language', 'value'),
        State('vocational', 'value'),
    ]
)
def upload_student_data(n_clicks, admission_number, roll_number, photo_id, class_no, section, current_year, enrolled,language,vocational):
    if n_clicks > 0:
        try:
            # Validate required fields
            if not all([admission_number, roll_number, photo_id, class_no, section, current_year, enrolled]):
                return "Please fill all required fields"
            
            # Convert enrolled to boolean
            currently_enrolled = True if enrolled == 'Yes' else False
            
            # Connect to the database
            conn = get_db_connection()
            cursor = conn.cursor()

            # Check if admission number already exists
            check_query = "SELECT COUNT(*) FROM ClassDetails WHERE admission_number = %s and year = %s"
            cursor.execute(check_query, (admission_number, current_year))
            count = cursor.fetchone()[0]

            if count > 0:
                return "Admission number or Aadhar number already exists. Please use unique values."

            # SQL query to insert data
            insert_query = """
                INSERT INTO ClassDetails (admission_number, year, class, section, roll_number, photo_id, currently_enrolled,language,vocational)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                ON CONFLICT (admission_number, year) DO UPDATE
                SET roll_number = EXCLUDED.roll_number,
                    photo_id = EXCLUDED.photo_id,
                    currently_enrolled = EXCLUDED.currently_enrolled,
                    language = EXCLUDED.language,
                    vocational = EXCLUDED.vocational
            """

            # Execute query with the submitted data
            cursor.execute(insert_query, (
                admission_number, current_year, class_no, section, roll_number, photo_id, currently_enrolled,language,vocational
            ))

            # Commit changes
            conn.commit()

            # Close connection
            cursor.close()
            conn.close()

            return "Data submitted successfully!"

        except Exception as e:
            return f"Error: {str(e)}"

    return "Submit"

In [690]:
# Define the callback to handle Fee Details form submission
@app.callback(
    Output('fee-details-submit-button', 'children'),
    inputs=[
        Input('fee-details-submit-button', 'n_clicks')
    ],
    state=[
        State('admission_number', 'value'),
        State('year', 'value'),
        State('school_fee', 'value'),
        State('school_fee_concession_reason', 'value'),
        State('transport_used', 'value'),
        State('transport_route', 'value'),
        State('transport_fee', 'value'),
        State('transport_fee_concession', 'value'),
        State('application_fee', 'value')
    ]
)
def submit_fee_details(n_clicks, admission_number, year, school_fee, school_fee_concession_reason, transport_used, transport_route, transport_fee, transport_fee_concession, application_fee):
    if n_clicks > 0:
        try:
            # Validate inputs
            if not all([admission_number, year, school_fee, application_fee]):
                return "Error: Required fields are missing."

            # Convert transport_used to boolean
            transport_used_boolean = True if transport_used == 'Yes' else False

            # Set transport-related fields to 0 if transport is not used
            if not transport_used_boolean:
                transport_fee = 0
                transport_fee_concession = 0
                transport_id = 0
            else:
                # Establish a database connection
                conn = get_db_connection()
                cursor = conn.cursor()

                # Check and get transport_id if transport is used
                transport_id = None
                if transport_route:
                    cursor.execute("SELECT transport_id FROM Transport WHERE route_number = %s", (transport_route,))
                    result = cursor.fetchone()
                    if result:
                        transport_id = result[0]
                    else:
                        return f"Error: No transport found for route number {transport_route}."

                # Close the cursor (will be reopened for the next query)
                cursor.close()
                conn.close()

            # Re-establish a database connection for inserting data
            conn = get_db_connection()
            cursor = conn.cursor()

            # Insert or update data into the Fee table
            insert_fee_query = """
                INSERT INTO Fee (
                    admission_number, year, school_fee, concession_reason, application_fee, transport_fee, transport_fee_concession, transport_used, transport_id
                )
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                ON CONFLICT (admission_number, year) DO UPDATE
                SET school_fee = EXCLUDED.school_fee,
                    concession_reason = EXCLUDED.concession_reason,
                    application_fee = EXCLUDED.application_fee,
                    transport_fee = EXCLUDED.transport_fee,
                    transport_fee_concession = EXCLUDED.transport_fee_concession,
                    transport_used = EXCLUDED.transport_used,
                    transport_id = EXCLUDED.transport_id;
            """

            cursor.execute(insert_fee_query, (
                admission_number, year, school_fee, school_fee_concession_reason, application_fee, transport_fee, transport_fee_concession, transport_used_boolean, transport_id
            ))

            # Commit the transaction and close the connection
            conn.commit()
            cursor.close()
            conn.close()

            return "Fee details successfully uploaded!"

        except Exception as e:
            return f"Error: {str(e)}"

    return "Submit"


In [691]:
@app.callback(
    Output('transport-details-submit-button', 'children'),
    inputs=[
        Input('transport-details-submit-button', 'n_clicks'),
        State('pickup_point', 'value'),
        State('transport_route', 'value'),
    ]
)
def upload_student_data(n_clicks, pick_up_point, route_number):
    if n_clicks > 0:
        try:
            # Validate required fields
            if not all([pick_up_point, route_number]):
                return "Please fill all required fields"

            # Connect to the database
            conn = get_db_connection()
            cursor = conn.cursor()

            # Check if admission number already exists
            check_query = "SELECT COUNT(*) FROM Transport WHERE route_number = %s"
            cursor.execute(check_query, (route_number,))
            count = cursor.fetchone()[0]

            if count > 0:
                return "Route Number already exists. Please use unique values."

            # SQL query to insert data
            insert_query = """
                INSERT INTO Transport (
                    pick_up_point, route_number
                )
                VALUES (%s, %s)
            """

            # Execute query with the submitted data
            cursor.execute(insert_query, (
                pick_up_point, route_number
            ))

            # Commit changes
            conn.commit()

            # Close connection
            cursor.close()
            conn.close()

            return "Data submitted successfully!"

        except Exception as e:
            return f"Error: {str(e)}"

    return "Submit"

In [692]:
# Define the callback to handle Fee Payment Details form submission
@app.callback(
    Output('fee-content', 'children'),
    inputs=[
        Input('feepayment-details-submit-button', 'n_clicks')
    ],
    state=[
        State('admission_number', 'value'),
        State('year', 'value'),
        State('fee_type', 'value'),
        State('fee_term', 'value'),
        State('paid', 'value'),
        State('due', 'value'),
        State('receipt_number', 'value'),
        State('paid_date', 'date')
    ]
)
def submit_fee_payment_details(n_clicks, admission_number, year, fee_type, fee_term, paid, due, receipt_no, paid_date):
    if n_clicks > 0:
        try:
            # Validate required fields
            if not all([admission_number, year, fee_type, fee_term, paid, due, receipt_no, paid_date]):
                return "Error: All fields are required."

            # Establish a database connection
            conn = get_db_connection()
            cursor = conn.cursor()

            # Insert or update data into the FeeBreakdown table
            insert_fee_breakdown_query = """
                INSERT INTO FeeBreakdown (
                    admission_number, year, fee_type, term, paid, due, receipt_no, fee_paid_date
                )
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                ON CONFLICT (admission_number, year, fee_type) DO UPDATE
                SET term = EXCLUDED.term,
                    paid = EXCLUDED.paid,
                    due = EXCLUDED.due,
                    receipt_no = EXCLUDED.receipt_no,
                    fee_paid_date = EXCLUDED.fee_paid_date;
            """

            cursor.execute(insert_fee_breakdown_query, (
                admission_number, year, fee_type, fee_term, paid, due, receipt_no, paid_date
            ))

            # Commit the transaction and close the connection
            conn.commit()
            cursor.close()
            conn.close()

            return "Fee payment details successfully uploaded!"

        except Exception as e:
            return f"Error: {str(e)}"

    return ""


In [693]:
@app.callback(
    Output('class-details-output', 'children'),
    [Input('fetch-class-details', 'n_clicks')],
    [State('class-dropdown', 'value'), State('year-dropdown', 'value')]
)
def fetch_class_details(n_clicks, selected_class, selected_year):
    if not n_clicks:
        return dash.no_update

    conn = None  # Ensure connection is initialized
    try:
        # Establish a database connection
        conn = get_db_connection()
        cursor = conn.cursor()

        # Parameterized query to prevent SQL injection
        query = """
            WITH class_data AS (
                SELECT
                    cd.class,
                    cd.section,
                    s.gender,
                    yr.total_fee_paid,
                    yr.total_fee_due
                FROM classdetails cd
                LEFT JOIN Student s ON cd.admission_number = s.admission_number
                LEFT JOIN YearlyRecord yr ON cd.admission_number = yr.admission_number AND cd.year = yr.year
                WHERE cd.class = %s AND cd.year = %s
            )
            SELECT
                COUNT(*) AS total_students,
                SUM(CASE WHEN section = 'A' THEN 1 ELSE 0 END) AS Sec_A,
                SUM(CASE WHEN section = 'B' THEN 1 ELSE 0 END) AS Sec_B,
                SUM(CASE WHEN section = 'C' THEN 1 ELSE 0 END) AS Sec_C,
                SUM(CASE WHEN gender = 'Boy' THEN 1 ELSE 0 END) AS total_male,
                SUM(CASE WHEN gender = 'Girl' THEN 1 ELSE 0 END) AS total_female,
                SUM(total_fee_paid) AS total_fee_paid,
                SUM(total_fee_due) AS total_fee_due
            FROM class_data;
        """
        cursor.execute(query, (selected_class, selected_year))
        result = cursor.fetchone()

        if not result:
            return "No data found for the selected class and year."

        # Unpack results and handle None values
        total_students, sec_a, sec_b, sec_c, total_male, total_female, total_fee_paid, total_fee_due = result
        total_fee_paid = total_fee_paid or 0  # Default to 0 if None
        total_fee_due = total_fee_due or 0    # Default to 0 if None

        # Calculate total sections
        total_sections = sum([bool(sec_a), bool(sec_b), bool(sec_c)])

        # Return data as a Div
        return html.Div([
            html.P(f"Total Students: {total_students}"),
            html.P(f"Sec A: {sec_a}"),
            html.P(f"Sec B: {sec_b}"),
            html.P(f"Sec C: {sec_c}"),
            html.P(f"Male Students: {total_male}"),
            html.P(f"Female Students: {total_female}"),
            html.P(f"Total Fee Paid: ₹{total_fee_paid:.2f}"),
            html.P(f"Total Fee Due: ₹{total_fee_due:.2f}")
        ])

    except Exception as e:
        if conn and not conn.closed:  # Rollback only if the connection is open
            conn.rollback()
        return f"An error occurred: {str(e)}"

    finally:
        if conn and not conn.closed:  # Close connection if it's open
            conn.close()


In [694]:
@app.callback(
    Output('student-details-output', 'children'),
    [Input('fetch-student-details', 'n_clicks')],
    [State('admission-number', 'value'), State('student-year-dropdown', 'value')]
)
def fetch_student_details(n_clicks, admission_number, selected_year):
    if not n_clicks:
        return dash.no_update

    if not admission_number or not selected_year:
        return "Please provide both Admission Number and Year."

    try:
        # Establish a database connection
        conn = get_db_connection()
        cursor = conn.cursor()

        query = """
            SELECT
                s.admission_number,
                s.aadhar_number,
                s.student_name,
                s.guardian_name,
                s.gender,
                s.date_of_birth,
                s.date_of_joining,
                s.contact_number,
                s.village,
                yr.year,
                SUM(fb.paid) FILTER (WHERE fb.fee_type = 'school_fee') AS total_school_fee_paid,
                SUM(fb.paid) FILTER (WHERE fb.fee_type = 'transport_fee') AS total_transport_fee_paid,
                SUM(fb.paid) FILTER (WHERE fb.fee_type = 'application_fee') AS total_application_fee_paid,
                SUM(fb.paid) AS total_fee_paid,
                SUM(fb.due) FILTER (WHERE fb.fee_type = 'school_fee') AS total_school_fee_due,
                SUM(fb.due) FILTER (WHERE fb.fee_type = 'transport_fee') AS total_transport_fee_due,
                SUM(fb.due) FILTER (WHERE fb.fee_type = 'application_fee') AS total_application_fee_due,
                SUM(fb.due) AS total_fee_due
            FROM Student s
            LEFT JOIN FeeBreakdown fb ON s.admission_number = fb.admission_number
            LEFT JOIN YearlyRecord yr ON s.admission_number = yr.admission_number AND fb.year = yr.year
            WHERE fb.admission_number = %s AND fb.year = %s
            GROUP BY s.admission_number, s.aadhar_number, s.student_name, s.guardian_name, s.gender,
                     s.date_of_birth, s.date_of_joining, s.contact_number, s.village, yr.year;
        """
        cursor.execute(query, (admission_number, selected_year))
        records = cursor.fetchall()

        if not records:
            return "No data found for the selected admission number and year."

        record = records[0]
        fields = [
            "Admission Number", "Aadhar Number", "Student Name", "Guardian Name", "Gender",
            "Date of Birth", "Date of Joining", "Contact Number", "Village", "Year",
            "Total School Fee Paid", "Total Transport Fee Paid", "Total Application Fee Paid",
            "Total Fee Paid", "Total School Fee Due", "Total Transport Fee Due",
            "Total Application Fee Due", "Total Fee Due"
        ]
        return html.Div([html.P(f"{field}: {value}") for field, value in zip(fields, record)])
    except Exception as e:
        return f"An error occurred: {str(e)}"
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()


In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True)